**Objective**:
1. Build a restaurant recommender

In [ ]:
import pandas as pd
data = pd.read_csv("restaurants_cleaned.csv")
data.head()

,Unnamed: 0,Ranking,Rating,Name,City,Cuisine Style,Price Range,Number of Reviews,Reviews
0,0,1.0,5.0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",Mid-range,136.0,"[['Just like home', 'A Warm Welcome to Wintry ..."
1,1,2.0,4.5,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",Splurge,812.0,"[['Great food and staff', 'just perfect'], ['0..."
2,2,3.0,4.5,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",Splurge,567.0,"[['Satisfaction', 'Delicious old school restau..."
3,3,4.0,5.0,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",Splurge,564.0,"[['True five star dinner', 'A superb evening o..."
4,4,5.0,4.5,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",Splurge,316.0,"[['Best meal.... EVER', 'super food experience..."


In [ ]:
data.shape

(74225, 9)

In [ ]:
data.isnull().sum()

Unnamed: 0           0
Ranking              0
Rating               0
Name                 0
City                 0
Cuisine Style        0
Price Range          0
Number of Reviews    0
Reviews              0
dtype: int64

In [ ]:
data.drop_duplicates(subset ="Name", keep = False, inplace = True)

In [ ]:
data.shape

(64377, 9)

**Recommender based on City, Cuisine style, Price**

In [ ]:
data.drop(columns=["Unnamed: 0"],inplace=True)

In [ ]:
data.head()

,Ranking,Rating,Name,City,Cuisine Style,Price Range,Number of Reviews,Reviews
0,1.0,5.0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",Mid-range,136.0,"[['Just like home', 'A Warm Welcome to Wintry ..."
1,2.0,4.5,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",Splurge,812.0,"[['Great food and staff', 'just perfect'], ['0..."
2,3.0,4.5,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",Splurge,567.0,"[['Satisfaction', 'Delicious old school restau..."
3,4.0,5.0,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",Splurge,564.0,"[['True five star dinner', 'A superb evening o..."
4,5.0,4.5,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",Splurge,316.0,"[['Best meal.... EVER', 'super food experience..."


In [ ]:
from collections import Counter
city_count = data["City"]
Counter(city_count)

Counter({'Amsterdam': 2188,
         'Athens': 1074,
         'Barcelona': 4544,
         'Berlin': 2972,
         'Bratislava': 393,
         'Brussels': 1684,
         'Budapest': 1500,
         'Copenhagen': 1113,
         'Dublin': 1252,
         'Edinburgh': 1216,
         'Geneva': 794,
         'Hamburg': 1038,
         'Helsinki': 665,
         'Krakow': 746,
         'Lisbon': 2226,
         'Ljubljana': 295,
         'London': 9368,
         'Luxembourg': 374,
         'Lyon': 1195,
         'Madrid': 3902,
         'Milan': 3305,
         'Munich': 1426,
         'Oporto': 965,
         'Oslo': 676,
         'Paris': 8005,
         'Prague': 2244,
         'Rome': 4218,
         'Stockholm': 1189,
         'Vienna': 1813,
         'Warsaw': 1066,
         'Zurich': 931})

In [ ]:
data_types = data.apply(lambda x: x.apply(lambda y: type(y)).value_counts())
data_types

,Ranking,Rating,Name,City,Cuisine Style,Price Range,Number of Reviews,Reviews
<class 'float'>,64377.0,64377.0,NaN,NaN,NaN,NaN,64377.0,NaN
<class 'str'>,NaN,NaN,64377.0,64377.0,64377.0,64377.0,NaN,64377.0


In [ ]:
type(data["Cuisine Style"][0])

str

In [ ]:
from ast import literal_eval
data["Cuisine Style"] = data["Cuisine Style"].apply(literal_eval)

In [ ]:
type(data["Cuisine Style"][0])

list

In [ ]:
data_r.columns

Index(['Name', 'City', 'Cuisine Style', 'Price Range', 'soup'], dtype='object')

In [ ]:
data_r.drop(columns=["Ranking","Rating","Number of Reviews","Reviews"],inplace=True)

In [ ]:
data_r.head()

In [ ]:
data_r["Cuisine Style"] = [" ".join(x) for x in data_r["Cuisine Style"]]

In [ ]:
data_r.head()

In [ ]:
#lowercase
features = ['City',"Cuisine Style","Price Range"]
for feature in features:
  data_r[feature] = data_r[feature].apply(lambda x:str.lower(x))

In [ ]:
#soupify
data_r['soup'] = data_r[['City',"Cuisine Style","Price Range"]].apply(lambda x:" ".join(x),axis=1)

In [ ]:
data_r["soup"].head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

count = CountVectorizer()
matrix = count.fit_transform(data_r['soup'])

In [ ]:
matrix.shape

In [ ]:
#due to memory limitations cosine similarity computed with 20k restaurants
cosine_sim = cosine_similarity(matrix[:20_000], matrix[:20_000])

In [ ]:
data = data_r.reset_index()
indices = pd.Series(data.index, index=data['Name'])

In [ ]:
def recommend_restaurants(Name, cosine_sim=cosine_sim):
    idx = indices[Name] #get the index of the restaurant

    sim_scores = list(enumerate(cosine_sim[idx]))# get similarity of restaurants to the one chosen

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # sort based on similarity

    sim_scores = sim_scores[1:11] # get 10 most similar ones, except for 0 which is the chosen restaurant itself
 
    restaurant_indices = [i[0] for i in sim_scores] # get the indices

    result_sim = data['Name'].iloc[restaurant_indices].copy() # return top 10 most similar ones from the original data file
    city_sim = data["City"].iloc[restaurant_indices].copy()
    cuisine_sim = data["Cuisine Style"].iloc[restaurant_indices].copy()
    price_sim = data["Price Range"].iloc[restaurant_indices].copy()

    cosine_similarity_score = [i[1] for i in sim_scores]
    recommendation_and_score = pd.DataFrame({"restaurant":result_sim.tolist(), #dataframe with similar restaurants and their cos similarity scores
                                             "sim_score":cosine_similarity_score,
                                             "city":city_sim.to_list(),
                                             "cuisine":cuisine_sim.to_list(),
                                             "price":price_sim.to_list()}) 

    return recommendation_and_score

In [ ]:
recommend_restaurants('Vezos')

,restaurant,sim_score,city,cuisine,price
0,Taverna Ellada Hamburg,0.942809,Hamburg,"[Mediterranean, European, Greek, Vegetarian Fr...",Mid-range
1,Restaurant Kouros,0.942809,Hamburg,"[Mediterranean, European, Greek, Vegetarian Fr...",Mid-range
2,Taverna Nostalgia,0.942809,Hamburg,"[Mediterranean, Greek, European, Vegetarian Fr...",Mid-range
3,Kypros,0.942809,Hamburg,"[Mediterranean, European, Greek, Vegetarian Fr...",Mid-range
4,Nascherei,0.942809,Hamburg,"[Mediterranean, European, Greek, Vegetarian Fr...",Mid-range
5,Taverna Omiros,0.942809,Hamburg,"[Mediterranean, Greek, European, Vegetarian Fr...",Mid-range
6,Griekse Taverna,0.888889,Amsterdam,"[Mediterranean, European, Greek, Seafood, Vege...",Mid-range
7,Aiolou 68,0.888889,Athens,"[Seafood, Mediterranean, European, Greek, Vege...",Mid-range
8,Casa Franco,0.888889,Hamburg,"[Seafood, Mediterranean, European, Spanish, Ve...",Mid-range
9,La Vela - Ristorante & Weinbar,0.888889,Hamburg,"[Italian, Seafood, Mediterranean, European, Ve...",Mid-range
